In [ ]:
# 1_UNet_Training.ipynb (CORRECTED)

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# --- CONFIGURATION ---
IMG_HEIGHT = 256
IMG_WIDTH = 256
CHANNELS = 3
DATA_PATH = '../data/train'
MODEL_SAVE_PATH = '../saved_models/unet_oil_spill.h5'

# --- CORRECTED: Loss Function and Metric Definitions ---

# Dice Coefficient (F1-Score) - Uses SOFT predictions for loss calculation,
# but uses hard thresholding for reporting METRIC value (for human readability).

def dice_coeff_metric(y_true, y_pred, smooth=1e-6):
    # This is the version used for reporting the metric (uses hard thresholding)
    y_pred_metric = K.cast(K.greater(y_pred, 0.5), K.floatx())
    intersection = K.sum(y_true * y_pred_metric, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred_metric, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth) / (union + smooth), axis=0)
    return dice

def iou_metric(y_true, y_pred, smooth=1e-6):
    # This is the version used for reporting the metric (uses hard thresholding)
    y_pred_metric = K.cast(K.greater(y_pred, 0.5), K.floatx())
    intersection = K.sum(K.abs(y_true * y_pred_metric), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3]) + K.sum(y_pred_metric,[1,2,3]) - intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou


# Dice Loss: CRITICAL FIX - Must use SOFT predictions for differentiability.
def dice_loss(y_true, y_pred, smooth=1e-6):
    # This function is for the loss, so it must use SOFT predictions (y_pred)
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth) / (union + smooth), axis=0)
    return 1 - dice

# --- 1. SMART DATA LOADING HELPER (No changes needed) ---
def load_data(path):
    # ... (function body remains the same as your previous response) ...
    images = []
    masks = []
    
    img_dir = os.path.join(path, 'images')
    mask_dir = os.path.join(path, 'labels') 
    
    print(f"Looking for images in: {img_dir}")
    print(f"Looking for masks in: {mask_dir}")

    if not os.path.exists(img_dir):
        raise FileNotFoundError(f"ERROR: Image folder not found at {img_dir}")
    
    if not os.path.exists(mask_dir):
        # Fallback for different folder naming conventions
        mask_dir_alt = os.path.join(path, 'masks')
        if os.path.exists(mask_dir_alt):
            print(f"Found 'masks' folder instead of 'labels'. Using that.")
            mask_dir = mask_dir_alt
        else:
            raise FileNotFoundError(f"ERROR: Mask folder not found at {mask_dir}")

    files = os.listdir(img_dir)[:500] 
    print(f"Found {len(files)} files. Loading...")

    for file_name in files:
        # Load Image
        img_path = os.path.join(img_dir, file_name)
        img = cv2.imread(img_path)
        
        if img is None:
            continue 
        
        # Find Mask
        mask_path = os.path.join(mask_dir, file_name)
        
        # Handle PNG/JPG mismatch
        if not os.path.exists(mask_path):
            file_name_png = os.path.splitext(file_name)[0] + ".png"
            mask_path = os.path.join(mask_dir, file_name_png)
            
        mask = cv2.imread(mask_path, 0) 
        
        if mask is None:
            # print(f"Skipping {file_name} - No mask found.")
            continue

        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
        
        images.append(img)
        masks.append(mask)
        
    images = np.array(images) / 255.0
    masks = np.array(masks) / 255.0
    masks = np.expand_dims(masks, axis=-1)
    
    return images, masks

# Load Data
try:
    X, y = load_data(DATA_PATH)
    print(f"Successfully loaded {len(X)} images and masks.")
except Exception as e:
    print(e)
    
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# --- 2. IMPROVED U-NET MODEL (Added Batch Normalization) ---
def improved_unet_model(n_classes=1, img_height=256, img_width=256, img_channels=3):
    inputs = Input((img_height, img_width, img_channels))
    
    # Helper function for Contraction Block
    def conv_block(input_tensor, n_filters, dropout_rate=0.1):
        x = Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal')(input_tensor)
        x = BatchNormalization()(x) 
        x = Activation('relu')(x)
        x = Dropout(dropout_rate)(x)
        
        x = Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x) 
        x = Activation('relu')(x)
        return x

    # Contraction Path
    c1 = conv_block(inputs, 16, 0.1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = conv_block(p1, 32, 0.1)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = conv_block(p2, 64, 0.2)
    p3 = MaxPooling2D((2, 2))(c3)
    
    # Bottleneck
    c4 = conv_block(p3, 128, 0.2)
    
    # Expansion Path
    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c3]) 
    c5 = conv_block(u5, 64)
    
    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c2]) 
    c6 = conv_block(u6, 32)
    
    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c1]) 
    c7 = conv_block(u7, 16)
    
    outputs = Conv2D(n_classes, (1, 1), activation='sigmoid')(c7)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

model = improved_unet_model()

# --- CORRECTED: Use dice_loss for loss and dice_coeff_metric/iou_metric for metrics ---
model.compile(optimizer=Adam(learning_rate=1e-4), 
              loss=dice_loss, 
              metrics=['accuracy', iou_metric, dice_coeff_metric])
model.summary()

# --- 3. TRAINING ---
print("Starting Training...")
# Retaining low batch size (1) due to potential CUDA/Memory issues flagged by the initial notebook.
# Epochs are kept high (30) for convergence time.
history = model.fit(
    X_train, 
    y_train, 
    batch_size=1, 
    epochs=30, 
    validation_data=(X_val, y_val)
)

# --- 4. SAVE MODEL ---
model.save(MODEL_SAVE_PATH)
print(f"Model saved at {MODEL_SAVE_PATH}")

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Dice Loss')
plt.legend()
plt.show()